In [3]:
!pip install boto3

     |████████████████████████████████| 133kB 4.1MB/s eta 0:00:01
     |████████████████████████████████| 71kB 5.4MB/s eta 0:00:011
     |████████████████████████████████| 5.7MB 5.6MB/s eta 0:00:01     |██████████▌                     | 1.9MB 5.6MB/s eta 0:00:01     |█████████████                   | 2.3MB 5.6MB/s eta 0:00:01     |████████████████                | 2.8MB 5.6MB/s eta 0:00:01
     |████████████████████████████████| 552kB 13.0MB/s eta 0:00:01     |███████████████████████████▌    | 471kB 13.0MB/s eta 0:00:01


In [5]:
import boto3
from datetime import datetime

# 1. Set variables

In [2]:
from keys.aws_keys import govcloud, govcloud_dev

In [12]:
todays_date = datetime.today().strftime('%m%d%Y')

# Source info
# source_access_key_id = govcloud['access_key_id']
# source_secret_access_key = govcloud['secret_access_key']
# source_region = 'us-gov-west-1'

# # Target info
target_access_key_id = govcloud_dev['access_key_id']
target_secret_access_key = govcloud_dev['secret_access_key']
target_region = 'us-gov-west-1'

target_account = 'blahblah'
target_subnet = 'subnet-blahblah'
target_security_group = ['sg-blahblah']

# List of instances
instance_list = ['blahblah-sa-test1',
                 'blahblah-sa-test2'] 

### 2. Establish source connection

In [13]:
session = boto3.Session(
    aws_access_key_id = target_access_key_id,
    aws_secret_access_key = target_secret_access_key
)

ec2 = session.resource('ec2', region_name=target_region)

In [14]:
def create_ec2(i):
    ec2.meta.client.run_instances(ImageId='ami-blahblah',
                                  InstanceType='t2.micro',
                                  KeyName='blahblah-sa-govcloud-key',
                                  MinCount=1,
                                  MaxCount=1,
                                  SubnetId=target_subnet,
                                  SecurityGroupIds=target_security_group,
                                  TagSpecifications=[
                                      {
                                          'ResourceType': 'instance',
                                          'Tags': [
                                              {
                                                  'Key': 'Name',
                                                  'Value': name
                                              }
                                          ]
                                      }
                                  ])
    print('Created EC2:', name, '\n')

In [15]:
for instance in instance_list:
    name = instance
    create_ec2(name)

Created EC2: jplis-sa-test1 

Created EC2: jplis-sa-test2 



In [23]:
def describe_ec2(i):
    """
    Get EC2 information and save them to a dictionary. This data is used to create the AMI and track information.
    """
    for tag in i.tags:
        if 'Name' in tag['Key']:
            name = tag['Value']
    ec2info[i.id] = {
        'Name': name,
        'ID': i.instance_id,
        'Type': i.instance_type,
        'Private IP': i.private_ip_address
    }
    print('Working on:\n', ec2info[i.id])

In [24]:
for instance in instance_list:
    describe_ec2(instance)

AttributeError: 'str' object has no attribute 'tags'

In [18]:
for instance in instance_list:
    print(instance)

jplis-sa-test1
jplis-sa-test2


In [31]:
tag_key = "Name"
# tag_value = ["jplis-sa-test1", "jplis-sa-test2"]

response = ec2.meta.client.describe_instances(
    Filters=[
        {
            'Name' : 'tag:'+tag_key,
            'Values': instance_list
        }
    ])

In [35]:
from pprint import pprint

In [37]:
pprint(response)

{'Reservations': [{'Groups': [],
                   'Instances': [{'AmiLaunchIndex': 0,
                                  'Architecture': 'x86_64',
                                  'BlockDeviceMappings': [{'DeviceName': '/dev/sda1',
                                                           'Ebs': {'AttachTime': datetime.datetime(2019, 8, 20, 22, 53, 31, tzinfo=tzlocal()),
                                                                   'DeleteOnTermination': True,
                                                                   'Status': 'attached',
                                                                   'VolumeId': 'vol-05f029b78f6a8a79b'}}],
                                  'CapacityReservationSpecification': {'CapacityReservationPreference': 'open'},
                                  'ClientToken': '',
                                  'CpuOptions': {'CoreCount': 1,
                                                 'ThreadsPerCore': 1},
                             

In [45]:
pprint(response['Reservations'])

[{'Groups': [],
  'Instances': [{'AmiLaunchIndex': 0,
                 'Architecture': 'x86_64',
                 'BlockDeviceMappings': [{'DeviceName': '/dev/sda1',
                                          'Ebs': {'AttachTime': datetime.datetime(2019, 8, 20, 22, 53, 31, tzinfo=tzlocal()),
                                                  'DeleteOnTermination': True,
                                                  'Status': 'attached',
                                                  'VolumeId': 'vol-05f029b78f6a8a79b'}}],
                 'CapacityReservationSpecification': {'CapacityReservationPreference': 'open'},
                 'ClientToken': '',
                 'CpuOptions': {'CoreCount': 1, 'ThreadsPerCore': 1},
                 'EbsOptimized': False,
                 'HibernationOptions': {'Configured': False},
                 'Hypervisor': 'xen',
                 'ImageId': 'ami-fee77d9f',
                 'InstanceId': 'i-08882b69bdaab076c',
                 'InstanceT

In [50]:
pprint(response['Reservations'][1]['Instances'])

[{'AmiLaunchIndex': 0,
  'Architecture': 'x86_64',
  'BlockDeviceMappings': [{'DeviceName': '/dev/sda1',
                           'Ebs': {'AttachTime': datetime.datetime(2019, 8, 20, 22, 53, 32, tzinfo=tzlocal()),
                                   'DeleteOnTermination': True,
                                   'Status': 'attached',
                                   'VolumeId': 'vol-04dc4abaea63019cc'}}],
  'CapacityReservationSpecification': {'CapacityReservationPreference': 'open'},
  'ClientToken': '',
  'CpuOptions': {'CoreCount': 1, 'ThreadsPerCore': 1},
  'EbsOptimized': False,
  'HibernationOptions': {'Configured': False},
  'Hypervisor': 'xen',
  'ImageId': 'ami-fee77d9f',
  'InstanceId': 'i-0e4664ecbec7e449f',
  'InstanceType': 't2.micro',
  'KeyName': 'jplit-sa-govcloud-key',
  'LaunchTime': datetime.datetime(2019, 8, 20, 22, 53, 32, tzinfo=tzlocal()),
  'Monitoring': {'State': 'disabled'},
  'NetworkInterfaces': [{'Attachment': {'AttachTime': datetime.datetime(2019, 8, 20

In [53]:
ec2_to_delete = []
for reservation in response['Reservations']:
 
    for instance in reservation['Instances']:
        id=instance["InstanceId"]
        instance_tag = instance['Tags']
        print ("Instance ID is %s" % id)
        ec2_to_delete.append(id)

Instance ID is i-08882b69bdaab076c
Instance ID is i-0e4664ecbec7e449f


In [55]:
ec2_to_delete

['i-08882b69bdaab076c', 'i-0e4664ecbec7e449f']

In [62]:
def delete_ec2(i):
    ec2.meta.client.terminate_instances(InstanceIds=i)
    print('Deleted EC2:', i, '\n')

In [64]:
delete_ec2(ec2_to_delete)

Deleted EC2: ['i-08882b69bdaab076c', 'i-0e4664ecbec7e449f'] 

